In [1]:
import pandas as pd 
import os 
import sys 
import pickle
from pycaret.classification import *

sys.path.append('../src')
from utils import getExperimentConfig, getPicklesFromDir, run_pycaret_setup

In [2]:
# Get global variables for the experiment
config = getExperimentConfig()
# Get folders
folders = config['folders']
# Load dataset specific settings (from the real-data)
dataset_settings = getPicklesFromDir(folders['settings_dir'])

In [3]:
dataset_settings[0]

{'meta': {'name': 'Diabetes',
  'id': 'D0',
  'filename': 'diabetes.csv',
  'target': 'Outcome',
  'ordinal_features': None,
  'numeric_features': ['DiabetesPedigreeFunction',
   'BMI',
   'Insulin',
   'Glucose',
   'Age',
   'SkinThickness',
   'BloodPressure',
   'Pregnancies'],
  'text_features': None,
  'categorical_features': None,
  'sd_meta_list': [{'id': 'SD0Q1_0',
    'path': '../data/synthetic/SD0Q1_0.csv',
    'sdg_params': {'epochs': 300, 'batch_size': 50}},
   {'id': 'SD0Q1_1',
    'path': '../data/synthetic/SD0Q1_1.csv',
    'sdg_params': {'epochs': 300, 'batch_size': 50}}]},
 'setup_param': {'target': 'Outcome',
  'train_size': 0.8,
  'fold_strategy': 'stratifiedkfold',
  'fold': 10,
  'ordinal_features': None,
  'numeric_features': ['DiabetesPedigreeFunction',
   'BMI',
   'Insulin',
   'Glucose',
   'Age',
   'SkinThickness',
   'BloodPressure',
   'Pregnancies'],
  'text_features': None,
  'categorical_features': None,
  'imputation_type': 'simple',
  'numeric_imputa

Psuedo code for how to test the synthetic datasets,


```
for each real_dataset in sd_meta:
    Get real_dataset settings and parameters
    
    in setup_param, set 'log_data' = false, to disable saving the train test split, no need to save it
    in setup_param update 'system_log' to S + current-value
    in setup_param, set 'experiment_custom_tags' to 'Synthetic'
    
    for each sd_meta_info in sd_meta[real_dataset]:
    
        update 'experiment_name' in in setup_param to the current sd_id
        
        run setup()
        for each ml in ml_models:
        
            clf = run create_model(ml)
            run tune_model(clf, optimize='Accuracy', n_iter)
            
            test?
        
    
    
```

In [4]:
for settings in dataset_settings:
    # update system_log name
    settings['setup_param']['system_log'] = folders['log_dir']+"Step4_SD"
    # disable saving train-test split data (to save space)
    settings['setup_param']['log_data'] = False

    for sd_meta in settings['meta']['sd_meta_list']:
        # Add custom tags to the logg, defining dataset type, Id and SDG hyperparameters
        settings['setup_param']['experiment_custom_tags'] = {
            'Dataset Type': 'Synthetic',
            'Dataset ID': sd_meta['id'],
            'SDG param': sd_meta['sdg_params']
        }

        run_pycaret_setup(sd_meta['path'], settings['setup_param'])

        for ml_model in config['clf']['ml_models']:
            # create then tune the model
            model = create_model(ml_model)
            tuned_model = tune_model(model, **config['clf']['tuning_param'])

,Description,Value
0,Session id,6988
1,Target,Outcome
2,Target type,Binary
3,Original data shape,"(768, 10)"
4,Transformed data shape,"(768, 10)"
5,Transformed train set shape,"(614, 10)"
6,Transformed test set shape,"(154, 10)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8065,0.7778,0.4118,0.7778,0.5385,0.4303,0.4652
1,0.6935,0.6275,0.2778,0.4545,0.3448,0.1598,0.1680
2,0.7742,0.7639,0.4444,0.6667,0.5333,0.3922,0.4062
3,0.7903,0.7740,0.3889,0.7778,0.5185,0.4030,0.4425
4,0.7541,0.7139,0.4118,0.5833,0.4828,0.3277,0.3363
5,0.8525,0.8489,0.5882,0.8333,0.6897,0.5966,0.6122
6,0.7869,0.6578,0.2941,0.8333,0.4348,0.3386,0.4086
7,0.7869,0.7861,0.2941,0.8333,0.4348,0.3386,0.4086
8,0.7705,0.7513,0.4706,0.6154,0.5333,0.3847,0.3908


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8065,0.7791,0.4118,0.7778,0.5385,0.4303,0.4652
1,0.6935,0.6263,0.2778,0.4545,0.3448,0.1598,0.1680
2,0.7742,0.7626,0.4444,0.6667,0.5333,0.3922,0.4062
3,0.7903,0.7727,0.3889,0.7778,0.5185,0.4030,0.4425
4,0.7377,0.7126,0.4118,0.5385,0.4667,0.2968,0.3015
5,0.8525,0.8489,0.5882,0.8333,0.6897,0.5966,0.6122
6,0.7869,0.6564,0.2941,0.8333,0.4348,0.3386,0.4086
7,0.8033,0.7861,0.3529,0.8571,0.5000,0.4029,0.4645
8,0.7705,0.7513,0.4706,0.6154,0.5333,0.3847,0.3908


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7903,0.7235,0.4118,0.7000,0.5185,0.3958,0.4186
1,0.6613,0.6465,0.1667,0.3333,0.2222,0.0356,0.0390
2,0.8065,0.7001,0.4444,0.8000,0.5714,0.4593,0.4924
3,0.7742,0.7923,0.3889,0.7000,0.5000,0.3692,0.3958
4,0.6721,0.5956,0.2941,0.3846,0.3333,0.1210,0.1230
5,0.7213,0.7574,0.3529,0.5000,0.4138,0.2381,0.2443
6,0.7705,0.6664,0.2941,0.7143,0.4167,0.3034,0.3498
7,0.8033,0.7152,0.4118,0.7778,0.5385,0.4281,0.4631
8,0.7377,0.6932,0.4706,0.5333,0.5000,0.3232,0.3243


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7903,0.8440,0.4667,0.5833,0.5185,0.3866,0.3905
1,0.8065,0.5858,0.4000,0.6667,0.5000,0.3892,0.4087
2,0.8226,0.7150,0.4286,0.6667,0.5217,0.4191,0.4345
3,0.7419,0.6339,0.4286,0.4286,0.4286,0.2619,0.2619
4,0.7213,0.6413,0.5000,0.4118,0.4516,0.2671,0.2694
5,0.7377,0.6444,0.3571,0.4167,0.3846,0.2192,0.2203
6,0.7213,0.6748,0.5000,0.4118,0.4516,0.2671,0.2694
7,0.7213,0.6535,0.2857,0.3636,0.3200,0.1479,0.1496
8,0.7377,0.6672,0.5000,0.4375,0.4667,0.2938,0.2949


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8065,0.8199,0.3333,0.7143,0.4545,0.3553,0.3935
1,0.7903,0.7560,0.2667,0.6667,0.3810,0.2816,0.3246
2,0.8065,0.7463,0.2857,0.6667,0.4000,0.3060,0.3451
3,0.8226,0.7798,0.4286,0.6667,0.5217,0.4191,0.4345
4,0.7541,0.7325,0.4286,0.4615,0.4444,0.2868,0.2872
5,0.7377,0.6596,0.1429,0.3333,0.2000,0.0722,0.0815
6,0.8525,0.8663,0.4286,0.8571,0.5714,0.4940,0.5374
7,0.7541,0.7401,0.2857,0.4444,0.3478,0.2050,0.2126
8,0.7869,0.7561,0.2143,0.6000,0.3158,0.2218,0.2633


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7903,0.8213,0.2000,0.7500,0.3158,0.2382,0.3116
1,0.7903,0.7674,0.2000,0.7500,0.3158,0.2382,0.3116
2,0.7903,0.7679,0.2143,0.6000,0.3158,0.2235,0.2651
3,0.8065,0.7857,0.2857,0.6667,0.4000,0.3060,0.3451
4,0.7541,0.7599,0.3571,0.4545,0.4000,0.2482,0.2510
5,0.7869,0.6976,0.2143,0.6000,0.3158,0.2218,0.2633
6,0.8361,0.9149,0.2857,1.0000,0.4444,0.3813,0.4854
7,0.7705,0.7644,0.2143,0.5000,0.3000,0.1882,0.2125
8,0.8197,0.7234,0.2143,1.0000,0.3529,0.2959,0.4167


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [6]:
"""
#update setup_param log setings
dataset_settings[0]['setup_param']['experiment_name'] = "SD0Q1_0"
dataset_settings[0]['setup_param']['system_log'] = folders['log_dir']+"Step4_SD"
# Do not save train test datasplit, to save resources
dataset_settings[0]['setup_param']['log_data'] = False
dataset_settings[0]['setup_param']['experiment_custom_tags'] = {'Dataset Type': 'Synthetic'}


pycaret_setup = setup(data=pd.read_csv(folders['sd_dir']+"SD0q1_0.csv"), 
                      **dataset_settings[0]['setup_param'])
"""

,Description,Value
0,Session id,6025
1,Target,Outcome
2,Target type,Binary
3,Original data shape,"(768, 10)"
4,Transformed data shape,"(768, 10)"
5,Transformed train set shape,"(614, 10)"
6,Transformed test set shape,"(154, 10)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


In [7]:
# the dataset specific saved experiment
#config_filename = folders['setup_dir']+'D0.pkl'

# load the experiment but with the synthetic data, and test_data=original_data.csv
#load_experiment(config_filename, data=pd.read_csv(folders['sd_dir']+"SD0Q1_0.csv"))
# Update log experiment name
#set_config('exp_name_log', 'SD0Q1_0')

In [8]:
#log = get_logs(exp_name)
#log.columns

In [ ]:
"""
best = compare_models(include=config['ml_models'])

# Dataset of performance with
tune_model(best, optimize='Accuracy', choose_better=True, n_iter=200)


model = load_model(folders['models_dir']+'D0')
"""

Following cell is example code on how to extract information from the mlflow logs

In [25]:
"""
exp_name =dataset_settings[0]['meta']['id']
#Get logs
get_logs(exp_name)[[
    'tags.mlflow.runName',
    'metrics.Accuracy',
    'metrics.F1',
    'metrics.Prec',
    'metrics.Recall',
    'tags.Source',
    'run_id', 
]].sort_values('metrics.Accuracy', ascending=False)
"""

,tags.mlflow.runName,metrics.Accuracy,metrics.F1,metrics.Prec,metrics.Recall,tags.Source,run_id
7,Random Forest Classifier,0.7900,0.6711,0.7465,0.6167,compare_models,a5d9b18b84d04269b0d11d3df3dde699
6,Logistic Regression,0.7754,0.6428,0.7319,0.5840,compare_models,8790ff71a4374e30a6297cd62365584c
1,Random Forest Classifier,0.7671,0.6981,0.6416,0.7749,tune_model,79492964faa146c8aa95bb2bfd4a6e6c
5,SVM - Radial Kernel,0.7655,0.6296,0.7207,0.5742,compare_models,656021909f5f48de9dcba758c3065bb0
4,Naive Bayes,0.7639,0.6462,0.6890,0.6203,compare_models,a1671c530fe144cea6b708ae50acaac6
3,K Neighbors Classifier,0.7558,0.6142,0.6864,0.5701,compare_models,c407633800fd46229a4cf6118f1334c7
2,SVM - Linear Kernel,0.7279,0.5689,0.6376,0.5238,compare_models,9e9d07ca556b452eb05165f67e599b98
0,Session Initialized 0dfa,NaN,NaN,NaN,NaN,setup,0ddcd068a0a24037971bfa78547b4270
8,Session Initialized 09bf,NaN,NaN,NaN,NaN,setup,8865811c66f74657b50760513f858724
